# Custom Image Classification

Determine Limits: https://learn.microsoft.com/en-us/azure/ai-services/custom-vision-service/limits-and-quotas

## Install Library

In [ ]:
%pip install azure-cognitiveservices-vision-customvision

## Load Azure Configuration

In [13]:
import os
# retrieve environment variables
training_endpoint = os.environ["AZURE_CUSTOM_VISION_TRAINING_ENDPOINT"]
training_key = os.environ["AZURE_CUSTOM_VISION_TRAINING_KEY"]
prediction_endpoint = os.environ["AZURE_CUSTOM_VISION_PREDICTION_ENDPOINT"]
prediction_key = os.environ["AZURE_CUSTOM_VISION_PREDICTION_KEY"]
prediction_resource_id = os.environ["AZURE_CUSTOM_VISION_PREDICTION_RESOURCE_ID"]

## Authenticate the client

In [ ]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import time, uuid

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(training_endpoint, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(prediction_endpoint, prediction_credentials)

## Create New Custom Vision Project

In [15]:
publish_iteration_name = "classifyModel"

# Create a new project
print ("Creating project...")
project_name = uuid.uuid4()
project = trainer.create_project(project_name)
print("Project created with id: {}".format(project.id))

Creating project...
Project created with id: 3a3aac96-24b7-4fee-8acd-9570dc05dc1c


## Add Tags to the Project

In [16]:
# Make two tags in the new project
hemlock_tag = trainer.create_tag(project.id, "Hemlock")
cherry_tag = trainer.create_tag(project.id, "Japanese Cherry")

## Upload and Tag Images

In [17]:
base_image_location = "../Data/customclassification/"

print("Adding images...")

image_list = []

for image_num in range(1, 11):
    file_name = "hemlock_{}.jpg".format(image_num)
    with open(os.path.join (base_image_location, "hemlock", file_name), "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[hemlock_tag.id]))

for image_num in range(1, 11):
    file_name = "japanese_cherry_{}.jpg".format(image_num)
    with open(os.path.join (base_image_location, "japanese_cherry", file_name), "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[cherry_tag.id]))

upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list))
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

Adding images...


## Train the project

In [18]:
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 10 seconds...")
    time.sleep(10)

Training...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Wa

## Publish the current iteration

In [19]:
# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!


## Test the prediction endpoint

In [20]:
# Use classify image of the CustomVisionPredictionClient to classify the test image
print ("Making a prediction...")
with open(os.path.join (base_image_location, "test/test_image.jpg"), "rb") as image_contents:
    results = predictor.classify_image(
        project.id, publish_iteration_name, image_contents.read())

    # Display the results.
    for prediction in results.predictions:
        print("\t" + prediction.tag_name +
              ": {0:.2f}%".format(prediction.probability * 100))

Making a prediction...
	Hemlock: 100.00%
	Japanese Cherry: 0.00%


## Further Predictions

Run this when you are using the trained model to make predictions for your application

In [22]:
import os
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

# retrieve environment variables
prediction_endpoint = os.environ["AZURE_CUSTOM_VISION_PREDICTION_ENDPOINT"]
prediction_key = os.environ["AZURE_CUSTOM_VISION_PREDICTION_KEY"]

# Create an instance of the prediction service
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(prediction_endpoint, prediction_credentials)

# Base Image Location
base_image_location = "../Data/customclassification/"

# Go to Custom Vision Studio to get the Project ID and Iteration Name
project_id = "3a3aac96-24b7-4fee-8acd-9570dc05dc1c"
iteration_name = "classifyModel"

# Use classify image of the CustomVisionPredictionClient to classify the test image
print ("Making a prediction...")
with open(os.path.join (base_image_location, "test/test_image.jpg"), "rb") as image_contents:
    results = predictor.classify_image(project_id, iteration_name, image_contents.read())

    # Display the results.
    for prediction in results.predictions:
        print("\t" + prediction.tag_name +
              ": {0:.2f}%".format(prediction.probability * 100))

Making a prediction...
	Hemlock: 100.00%
	Japanese Cherry: 0.00%


## Delete Resources

In [ ]:
print("Cleaning up...")
# unpublish iteration, delete the iteration, then delete the project
trainer.unpublish_iteration(project.id, iteration.id)
trainer.delete_iteration(project.id, iteration.id)
trainer.delete_project(project.id)


Cleaning up...
